# Question Answering

Wie du selbst gesehen hast, lassen sich BERT bzw. die davon abgeleiteten Transfer-Learning-Modelle leicht mithilfe des sog. *Finetunings* an Spezialaufgaben anpassen.

Eine sehr spezielle Aufgabe ist das sog. *Question Answering*. Dabei wird das Modell auf einen Korpus angepasst, der Dokumente, Fragen zu diesen Dokumenten und die jeweiligen Antworten (in den Dokumenten) enthält. Das passiert mit der sog. *Next Sentence Prediction*.

Wenn du solche Modelle nutzt, kannst du dir zu einem vorhandenen Dokument und einer Frage die richtig Antwort vorhersagen lassen.

## Modell vorbereiten

Für Question Answering brauchst du ein eigenes Modell. Am besten legst du das als Pipeline an, damit das Modell gleich richtig tokenisieren und vorhersagen kann:

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

model_name = "Sahajtomar/German-question-answer-Electra"
qa = pipeline("question-answering", model=model_name,
              tokenizer=model_name, device=0)

## Fragen zu Heise-Artikeln beanworten

Am besten beginnst du mit einem Artikel aus dem Newsticker und versuchst, ob das Modell dir dazu Fragen beantworten kann:

In [ ]:
text = """Apple-Chef fordert klimaneutrale Wirtschaft

Die "Zukunft des Planeten" und Profit schließen sich nicht aus, 
meint Tim Cook –  Konzerne und Regierungen sollten endlich die Kurve 
beim Klimaschutz kriegen.

Apple-Chef Tim Cook hat Regierungen und Unternehmen dazu aufgerufen, 
sich gegen den Klimawandel zu stemmen. In diesem Moment "historischer 
Dringlichkeit" müssten Konzerne endlich handeln, betonte der Apple-Chef 
auf dem Climate Ambition Summit 2020, den die Vereinten Nationen gemeinsam 
mit mehreren Ländern ausrichteten. Apple sei es bei den eigenen Einrichtungen 
bereits gelungen, klimaneutral zu arbeiten. Auch der Ausstoß klimaschädlichen
Kohlenstoffdioxids bei den Zulieferern soll drastisch reduziert werden.

Man helfe aktuell 95 Zulieferern dabei, komplett auf erneuerbare Energien
umzusatteln, so Cook. Das Unternehmen will bis 2030 vollständig klimaneutral 
arbeiten – die riesige Lieferkette eingerechnet.

Man müsse sich nicht zwischen Unternehmensgewinnen und "der Zukunft des 
Planeten" entscheiden, erläuterte Cook, "jede neue grüne Innovation" sei 
der Beweis, das sich beides vereinbaren lässt. Der CEO rief andere Unternehmen 
dazu auf, 2021 beim Klimaschutz "die Kurve zu kriegen". Es sei möglich, 
gemeinsam den Übergang zu einer klimaneutralen Wirtschaft zu erreichen – 
wenn nicht länger nur "an den Rändern" herumgedoktort wird. Apple wolle 
ein "Partner" für diesen Übergang sein.

Großbritannien, das den Klimagipfel mit ausrichtete, hatte Apple zuletzt 
wegen seines Geräte-Designs und der Reparaturvorgaben scharf kritisiert: 
Der iPhone-Hersteller sorge unnötig für zusätzlichen Elektroschrott durch 
immer schwerer zu reparierende Geräte, bei denen der Neukauf oft attraktiver 
als die Instandsetzung sei, beklagten britische Abgeordnete des 
Umweltausschusses im britischen Unterhaus. Apple zeigte sich "enttäuscht" 
über den Bericht, für Kunden gebe es mehr Reparatur- und Recycling-Optionen 
denn je.

Apple will recyclete iPhones
Cook betonte auf dem Klimagipfel auch, dass es Apples Ziel bleibe, keine 
neuen Rohstoffe mehr für die Produktion von Hardware fördern zu müssen, 
sondern in allen Bereichen auf Recycling zu setzen. Ein Zeitziel wurde 
hier nicht genannt. Apple betont seit längerem, saubere Energie könne 
langfristig auch wirtschaftlich sinnvoller für Konzerne sein. Andere 
US-IT-Riesen haben ebenfalls Klimaneutralität in Aussicht gestellt: 
Microsoft zielt ebenfalls auf das Jahr 2030 ab, Amazon auf 2040.
"""

Beginnen wir einfach:

In [ ]:
answer = qa(question="Wer ist der Chef von Apple?", context=text)
print(f"Answer: '{answer['answer']}', Score {answer['score']}")

Das ist schon ziemlich beeindruckend. Versuche eine allgemeinere Frage:

In [ ]:
answer = qa(question="Was will Apple?", context=text)
print(f"Answer: '{answer['answer']}', Score {answer['score']}")

Das ist sehr auf den Punkt gebracht. Mache es noch etwas schwieriger:

In [ ]:
answer = qa(question="Wer richtete den Klimagipfel aus?", context=text)
print(f"Answer: '{answer['answer']}', Score {answer['score']}")

Wow, das ist schon wirklich gut!

## Fragen zu Wikipedia beantworten

Steige nun in den nächsten Level ein und nutze einen komplizierteren Text, nämlich einen Wikipedia Artikel über [Die Simpsons](https://de.wikipedia.org/wiki/Die_Simpsons).

In [ ]:
!pip install readability-lxml

In [ ]:
from readability import Document
import requests
from bs4 import BeautifulSoup
doc = Document(requests.get("https://de.wikipedia.org/wiki/Die_Simpsons", stream=True).text)
soup = BeautifulSoup(doc.summary())            
content = soup.text
len(content)

Das sind immerhin fast 100 kB! Versuche nun, einfache Fragen dazu zu stellen:

In [ ]:
answer = qa(question="Wie heißt der Bruder von Lisa?", context=content)
print(f"Answer: '{answer['answer']}', Score {answer['score']}")

Das hat auch wieder sehr gut geklappt. Probiere, ob es auch mit etwas komplizierteren Fragen klappt:

In [ ]:
answer = qa(question="Wie heißen die Schwestern von Bart?", context=content)
print(f"Answer: '{answer['answer']}', Score {answer['score']}")

Auch sehr gut!

Gehe nun weiter in die *Details der Simpsons*:

In [ ]:
answer = qa(question="Wie heißen die Schwestern von Marge?", context=content)
print(f"Answer: '{answer['answer']}', Score {answer['score']}")

Das stimmt leider nicht. Zwar stehen *Patty* und *Selma* in dem Wikipedia-Artikel drin, aber sie werden nicht als Marges Schwestern genannt. So hat das Modell wohl Marges Kinder ausgewählt - immerhin ist das richtig.

Teste nun noch eine andere Datenquelle, nämlich [Raumschiff Enterprise](https://de.wikipedia.org/wiki/Raumschiff_Enterprise).

In [ ]:
doc = Document(requests.get("https://de.wikipedia.org/wiki/Raumschiff_Enterprise", stream=True).text)
soup = BeautifulSoup(doc.summary())            
content = soup.text
len(content)

Der Artikel ist etwas kürzer, versuche es auch hier mit Fragen:

In [ ]:
answer = qa(question="Wer spielt Spock?", context=content)
print(f"Answer: '{answer['answer']}' with score {answer['score']}")

Super! Geht es so gut weiter?

In [ ]:
answer = qa(question="Wer ist der Captain der Enterprise?", context=content)

print(f"Answer: '{answer['answer']}' with score {answer['score']}")

Leider nicht ganz. Das ist der Schauspieler von Captain Kirk.

## Spaß mit Question Answering

Fragen zu Texten beantworten zu lassen macht definitiv *Spaß*. Du musst nicht mehr den ganzen Text lesen und hast das Gefühl, dass der Computer dir wirklich Arbeit abnimmt. Das ist super!

Die Anwendungsfälle sind allerdings noch nicht ganz so klar. Ein Chatbot könnte das gut verwenden, aber muss auf eine viel größere Datenbasis zugreifen, um überhaupt die richtigen Artikel zu finden. Oftmals kannst du mit einer gezielten Suche auch gute Ergebnisse erreichen und musst dir nicht immer Fragen überlegen.

Zweifellos ist die Technologie aber sehr beeindruckend!